In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #conexão com o google drive (onde está o banco de dados)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/IA-projeto/BERT/BERT_DF.xlsx')
df

,Unnamed: 0,TEXT,RESULTADO,uppercase_percentage,special_char_count,contagem_palavras_filtradas
0,1,Santa Catarina Número de cidades em emergênci...,0,0.066246,0,0
1,2,Irã Presidente era conhecido como carniceiro...,0,0.058005,0,0
2,3,Corinthians fala em negociação legal após denú...,0,0.069182,0,2
3,5,Segundo hospital de campanha é aberto em Porto...,0,0.074468,0,0
4,6,Luladrão gastou MILHÕES visitando monstros di...,1,0.135593,1,0
...,...,...,...,...,...,...
77,94,Eliminação de judoca brasileiro nas Olimpíadas...,0,0.042683,0,1
78,95,Estudo inédito mostra que dos pacientes com ...,0,0.092105,0,0
79,96,Os Efeitos da Elevação do Nível do Mar nas Pra...,0,0.144444,0,0
80,97,Sócios do São Paulo barram mudança que previa ...,0,0.032680,0,0


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Carregar o modelo BERT e o tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Função para extrair embeddings do BERT e reduzir para 2D
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    # Pegando a média ao longo da dimensão da sequência para reduzir de 3D para 2D
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings.squeeze()

# Aplicando aos textos do dataframe
df['bert_embeddings'] = df['TEXT'].apply(get_bert_embeddings)

# Combinar as features (empilhando as colunas dos embeddings e das features adicionais)
X = np.hstack([np.stack(df['bert_embeddings'].values),
               df[['uppercase_percentage', 'special_char_count', 'contagem_palavras_filtradas']].values])
y = df['RESULTADO']

# Separar em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo SVM
model = SVC(kernel='linear', C=10.0)
model.fit(X_train, y_train)

# Prever e calcular acurácia
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy}")
#76% foi o maximo

Acurácia: 0.7647058823529411


In [23]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Carregar o modelo e tokenizer do BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Verificar se a GPU está disponível e mover o modelo para o dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Definindo o dataset customizado
class FakeNewsDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        inputs = tokenizer(self.texts[idx], return_tensors='pt', truncation=True, padding='max_length', max_length=256)
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}  # remove dimension [1, x] -> [x]
        inputs['labels'] = torch.tensor(self.labels[idx])
        return inputs

# Função de collate para ajustar o tamanho dos lotes
def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    return {'input_ids': input_ids.to(device),
            'attention_mask': attention_mask.to(device),
            'labels': labels.to(device)}

# Preparar os dados
X_train, X_test, y_train, y_test = train_test_split(df['TEXT'], df['RESULTADO'], test_size=0.2, random_state=42)

# Criar datasets
train_dataset = FakeNewsDataset(X_train.tolist(), y_train.tolist())
test_dataset = FakeNewsDataset(X_test.tolist(), y_test.tolist())

# DataLoader com collate_fn
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Função de treinamento
def train_and_save_best_model(model, dataloader, optimizer, num_epochs=3):
    best_accuracy = 0
    best_model_path = 'melhor_modelo_bert.pkl'

    for epoch in range(num_epochs):
        model.train()
        for batch in dataloader:
            optimizer.zero_grad()
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
            loss = outputs.loss
            loss.backward()
            optimizer.step()

        # Avaliação após cada época
        accuracy = evaluate(model, test_loader)
        print(f"Época {epoch + 1}, Acurácia: {accuracy}")

        # Salvar o modelo com a melhor acurácia
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), best_model_path)
            print(f"Novo melhor modelo salvo em '{best_model_path}' com acurácia: {best_accuracy}")

    return model

# Função de avaliação
def evaluate(model, dataloader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
            logits = outputs.logits
            predictions.append(torch.argmax(logits, dim=1).cpu().numpy())
            true_labels.append(batch['labels'].cpu().numpy())
    return accuracy_score(np.concatenate(true_labels), np.concatenate(predictions))

# Treinamento e salvamento do melhor modelo
train_and_save_best_model(model, train_loader, optimizer)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Época 1, Acurácia: 0.7647058823529411
Novo melhor modelo salvo em 'melhor_modelo_bert.pkl' com acurácia: 0.7647058823529411
Época 2, Acurácia: 0.8235294117647058
Novo melhor modelo salvo em 'melhor_modelo_bert.pkl' com acurácia: 0.8235294117647058
Época 3, Acurácia: 0.5882352941176471


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [25]:
tokenizer.save_pretrained('/content/drive/MyDrive/IA-projeto/aplicação')

('/content/drive/MyDrive/IA-projeto/aplicação/tokenizer_config.json',
 '/content/drive/MyDrive/IA-projeto/aplicação/special_tokens_map.json',
 '/content/drive/MyDrive/IA-projeto/aplicação/vocab.txt',
 '/content/drive/MyDrive/IA-projeto/aplicação/added_tokens.json')

In [16]:
'''import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Carregar o modelo e tokenizer do BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Verificar se a GPU está disponível e mover o modelo para o dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Definindo o dataset customizado
class FakeNewsDataset(Dataset):
    def __init__(self, texts, features, labels):
        self.texts = texts
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        inputs = tokenizer(self.texts[idx], return_tensors='pt', truncation=True, padding='max_length', max_length=256)
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}  # Remove a dimensão [1, x] -> [x]
        inputs['additional_features'] = torch.tensor(self.features[idx], dtype=torch.float)
        inputs['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return inputs

# Função de collate para ajustar o tamanho dos lotes
def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    additional_features = torch.stack([item['additional_features'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    return {
        'input_ids': input_ids.to(device),
        'attention_mask': attention_mask.to(device),
        'additional_features': additional_features.to(device),
        'labels': labels.to(device)
    }

# Preparar os dados
X = df['TEXT'].values
additional_features = df[['uppercase_percentage', 'special_char_count', 'contagem_palavras_filtradas']].values
y = df['RESULTADO'].values

# Separar em treino e teste
X_train, X_test, features_train, features_test, y_train, y_test = train_test_split(
    X, additional_features, y, test_size=0.2, random_state=42
)

# Criar datasets
train_dataset = FakeNewsDataset(X_train, features_train, y_train)
test_dataset = FakeNewsDataset(X_test, features_test, y_test)

# DataLoader com collate_fn
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Função de treinamento
def train(model, dataloader, optimizer):
    model.train()
    for batch in dataloader:
        optimizer.zero_grad()
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Função de avaliação
def evaluate(model, dataloader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
            logits = outputs.logits
            predictions.append(torch.argmax(logits, dim=1).cpu().numpy())
            true_labels.append(batch['labels'].cpu().numpy())
    return accuracy_score(np.concatenate(true_labels), np.concatenate(predictions))

# Treinamento
for epoch in range(3):  # Ajuste o número de épocas conforme necessário
    train(model, train_loader, optimizer)
    accuracy = evaluate(model, test_loader)
    print(f"Época {epoch + 1}, Acurácia: {accuracy}")

# Avaliação final
accuracy = evaluate(model, test_loader)
print(f"Acurácia final: {accuracy}")
'''

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Época 1, Acurácia: 0.5882352941176471
Época 2, Acurácia: 0.5294117647058824
Época 3, Acurácia: 0.6470588235294118
Acurácia final: 0.6470588235294118
